In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.lines import Line2D
import matplotlib

from scipy.stats import gamma
from scipy.stats import norm
from scipy.stats import multivariate_normal
from scipy.stats import gaussian_kde
from scipy.special import expit
from scipy.special import logsumexp
from scipy.optimize import minimize
from scipy.optimize import LinearConstraint
from scipy.signal import convolve
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter1d

import divebomb

import time
import pickle

import importlib

matplotlib.rcParams.update({'font.size': 12})
#matplotlib.rcParams.update({'lines.markersize': 20})

np.random.seed(0)

import Preprocessor
import Parameters
import HHMM
import Visualisor

train_new = False
#model = 'CarHMM'
#model = 'HHMM'
#model = 'CarHHMM1'
model = 'CarHHMM2'

In [ ]:
pars = Parameters.Parameters()

pars.cvc_file = '../Dat/diary/20200826-140823-Gikumi_2020_OG_137.cvc'
pars.csv_file = '../Dat/diary/20200826-140823-Gikumi_2020_OG_137.csv'

pars.stime = '2020-08-25 10:20:00'
pars.etime = '2020-08-25 15:45:00'
pars.drop_times = []


pars.features = [{'dive_duration':{'corr':False,'f':'gamma'}},
                 {'Ax':{'corr':True,'f':'normal'},
                  'Ay':{'corr':True,'f':'normal'},
                  'Az':{'corr':True,'f':'normal'},
                  'Ahat_low':{'thresh':5,'corr':False,'f':'gamma'}}]

pars.K = [2,3]
if model == 'CarHMM':
    pars.K = [1,3]
elif model == 'HHMM':
    pars.features[1]['Ax']['corr'] = False
    pars.features[1]['Ay']['corr'] = False
    pars.features[1]['Az']['corr'] = False
elif model == 'CarHHMM1':
    pars.features[1] = {'Ax':{'corr':True,'f':'normal'},
                        'Ay':{'corr':True,'f':'normal'},
                        'Az':{'corr':True,'f':'normal'}}

prep = Preprocessor.Preprocessor(pars)

df = prep.load_data(pars.cvc_file,
                    pars.csv_file,
                    pars.cvc_cols)

In [ ]:
df['elevation'] = -df['depth']
df['VeDBA'] = np.sqrt(df['Ax']**2 + df['Ay']**2 + df['Az']**2)
df['jerk'] = 50*gaussian_filter1d(df['VeDBA'], sigma=1, order=1, mode='wrap')

In [ ]:
# load in ethogram
ethogram_file = '../Dat/ethograms/I107_ethogram_Dec_14_2020.csv'
ethogram = pd.read_csv(ethogram_file,encoding='cp1252')

ethogram['time'] = pd.to_datetime('2020-08-25 ' + ethogram['event.time'],
                                   format = '%Y-%m-%d %I:%M:%S %p',
                                   errors = 'coerce')

In [ ]:
pd.set_option('display.width', 10000)
pd.set_option('max_colwidth', 600)
crunches = ethogram[ethogram['Behavior'] == 'quick']
display(crunches)

In [ ]:
ethogram['Media file path'][0]

In [ ]:
#stime = '2020-08-25 12:30:00'
#etime = '2020-08-25 12:35:00'
stimes0 = ['2020-08-25 00:00:00','2020-08-25 12:00:00','2020-08-25 13:00:00','2020-08-25 14:00:00','2020-08-25 15:00:00']
etimes0 = ['2020-08-26 14:00:00','2020-08-25 13:00:00','2020-08-25 14:00:00','2020-08-25 15:00:00','2020-08-25 16:00:00']
stimes = ['2020-08-25 10:00:00','2020-08-25 12:05:00','2020-08-25 12:18:00',
          '2020-08-25 12:22:00','2020-08-25 12:26:00',
          '2020-08-25 12:41:00','2020-08-25 13:20:00',
          '2020-08-25 13:26:00','2020-08-25 13:37:00'] + stimes0
etimes = ['2020-08-26 01:00:00','2020-08-25 12:13:00','2020-08-25 12:21:00',
          '2020-08-25 12:25:30','2020-08-25 12:34:00',
          '2020-08-25 12:47:00','2020-08-25 13:26:00',
          '2020-08-25 13:31:00','2020-08-25 13:45:00'] + etimes0

for stime,etime in zip(stimes,etimes):
    
    if stime == '2020-08-25 00:00:00':
        cmap = cm.get_cmap('tab10')
    else:
        cmap = cm.get_cmap('tab10')

    cols = ['elevation',
            'roll','pitch','yaw',
            'Ax','Ay','Az','VeDBA','jerk',
            'Mx','My','Mz',
            'gyro x','gyro y','gyro z']

    # plot data
    fig,ax = plt.subplots(nrows=len(cols),ncols=1,figsize = (20,50))
    crunch_df = df[(df['time'] > stime) &\
                   (df['time'] < etime)].iloc[::10]

    for i,col in enumerate(cols):
        ax[i].plot(crunch_df['time'],crunch_df[col],color='k',linewidth=0.5)
        ax[i].set_ylabel(col)


    # plot behaviors
    point_ethogram = ethogram[(ethogram['time'] > stime) &\
                              (ethogram['time'] < etime)]

    print(point_ethogram['Behavior'].unique())
    behaviors = np.array(['crunch','Scales','Echolocation','quick','Other whale with fish','bottom'])#point_ethogram['Behavior'].unique()
    for _,item in point_ethogram.iterrows():
        if item['Behavior'] in behaviors:
            try:
                color_ind = np.where(behaviors == item['Behavior'])[0][0]
                for i,col in enumerate(cols):
                    ax[i].axvline(item['time'],color=cmap(color_ind))
            except:
                print(item['Behavior'])
                print(behaviors)

    # make legend
    custom_lines = [Line2D([0], [0], color=cmap(i), lw=2) for i in range(len(behaviors))]
    fig.legend(custom_lines,behaviors)
    plt.show()

In [ ]:
df.columns